In [1]:
import yfinance as yf

import numpy as np
from numpy.linalg import multi_dot

np.random.seed(42)

import pandas as pd
pd.set_option('display.precision',4)

from datetime import datetime

# Import plotly express for EF plot
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
px.defaults.width, px.defaults.height = 1000,600
pio.templates.default = "plotly_dark"

import cufflinks as cf
cf.set_config_file(offline=True, dimensions=(1000,600),theme='space')

Download data

In [2]:
assets = ['STMMI','LDO','TRN','UCG','ENI']
assets.sort()
# Get yahoo tickers
yahooticker = [x+'.MI' for x in assets]

# Fetch data for multiple stocks at once
df = yf.download(yahooticker,start='2011-01-01')['Adj Close']
df[:'2019']

[*********************100%***********************]  5 of 5 completed


,ENI.MI,LDO.MI,STMMI.MI,TRN.MI,UCG.MI
Date,,,,,
2011-01-03,7.5742,7.5366,5.1224,1.7075,36.0565
2011-01-04,7.6710,7.4403,5.1384,1.6994,36.1930
2011-01-05,7.7725,7.4709,5.2626,1.7008,35.9426
2011-01-06,7.9247,7.6548,5.3550,1.6861,35.3963
2011-01-07,7.9109,7.6154,5.3422,1.6968,34.8273
...,...,...,...,...,...
2019-12-19,10.9671,10.1208,23.5306,5.0785,10.8293
2019-12-20,11.0684,10.1446,23.7747,5.1715,10.8020
2019-12-23,11.1017,10.2589,23.6965,5.1232,10.7057


In [3]:
# Plot price history
df.iplot(kind='line')

Compute overall annualized returns and volatility

In [4]:
returns = df.pct_change().dropna()
tradingDaysPerYear = 252
annualReturns = returns.mean()*tradingDaysPerYear
annualVolatility = returns.std()*np.sqrt(tradingDaysPerYear)

market_statistics = pd.DataFrame({
    'Average annual return': annualReturns,
    'Average annual volatility': annualVolatility
})
market_statistics

,Average annual return,Average annual volatility
ENI.MI,0.0889,0.2737
LDO.MI,0.1259,0.4022
STMMI.MI,0.2467,0.3971
TRN.MI,0.1403,0.2152
UCG.MI,0.0698,0.4620


In [5]:
yearStart = '2011'
yearEnd = '2020'
partial_returns = df[yearStart:yearEnd].pct_change().dropna()
partial_annualReturns = partial_returns.mean()*tradingDaysPerYear
partial_annualVolatility = partial_returns.std()*np.sqrt(tradingDaysPerYear)

partial_statistics = pd.DataFrame({
    'Average annual return': partial_annualReturns,
    'Average annual volatility': partial_annualVolatility
})
partial_statistics

,Average annual return,Average annual volatility
ENI.MI,0.0348,0.2782
LDO.MI,0.0622,0.4203
STMMI.MI,0.2600,0.4089
TRN.MI,0.1418,0.2170
UCG.MI,-0.0519,0.4771


In [6]:
yearStart = '2021'
yearEnd = '2023'
realized_returns = df[yearStart:yearEnd].pct_change().dropna()
realized_annualReturns = realized_returns.mean()*tradingDaysPerYear
realized_annualVolatility = realized_returns.std()*np.sqrt(tradingDaysPerYear)

realized_statistics = pd.DataFrame({
    'Average annual return': realized_annualReturns,
    'Average annual volatility': realized_annualVolatility
})
realized_statistics

,Average annual return,Average annual volatility
ENI.MI,0.2946,0.2562
LDO.MI,0.3682,0.3263
STMMI.MI,0.1837,0.3497
TRN.MI,0.1360,0.2089
UCG.MI,0.5257,0.4003


Add plots to visualize performance (stock price and volatility)

In [7]:
def MC_portfolioGeneration(returns, numPortfolios):
    # Number of assets in the market
    numAssets = len(returns.columns)

    # Initialize the lists
    rets, vols, wts = [],[],[]

    # Simulate 5,000 portfolios
    for i in range(numPortfolios):

        # Generate random weights
        weights = np.random.random(numAssets)

        # Normalize to 1 the sum of the weights
        weights /= np.sum(weights)

        # Portfolio statistics
        rets.append(weights.T @ np.array(returns.mean()*252))
        vols.append(np.sqrt(multi_dot([weights.T,returns.cov()*252,weights])))
        wts.append(weights)
        
    # Create a dataframe for analysis
    data = {'port_rets': rets, 'port_vols': vols}
    for counter, symbol in enumerate(returns.columns.to_list()):
        data[symbol+' weight'] = [w[counter] for w in wts]

    portdf = pd.DataFrame(data)
    portdf['sharpe_ratio'] = portdf['port_rets']/portdf['port_vols']

    return round(portdf,4)

In [8]:
# Number of portfolios
numPortfolios = 10000

portfolioSamples = MC_portfolioGeneration(partial_returns, numPortfolios)
portfolioSamples

,port_rets,port_vols,ENI.MI weight,LDO.MI weight,STMMI.MI weight,TRN.MI weight,UCG.MI weight,sharpe_ratio
0,0.1206,0.2791,0.1332,0.3381,0.2603,0.2129,0.0555,0.4323
1,0.1183,0.2936,0.0653,0.0243,0.3625,0.2516,0.2963,0.4029
2,0.1344,0.3226,0.0093,0.4375,0.3755,0.0958,0.0820,0.4168
3,0.1198,0.2754,0.1057,0.1753,0.3024,0.2489,0.1678,0.4348
4,0.0719,0.2716,0.3279,0.0748,0.1566,0.1963,0.2444,0.2647
...,...,...,...,...,...,...,...,...
9995,0.1077,0.2835,0.1597,0.1650,0.2952,0.1800,0.2000,0.3799
9996,0.1106,0.2658,0.0900,0.0450,0.2535,0.3642,0.2473,0.4162
9997,0.0653,0.3138,0.1866,0.2437,0.1976,0.0598,0.3124,0.2079
9998,0.0929,0.2869,0.2466,0.1848,0.2589,0.1116,0.1980,0.3238


In [25]:
# Plot simulated portfolio
fig = px.scatter(
    portfolioSamples, x='port_vols', y='port_rets', color='sharpe_ratio',
    labels={'port_vols': 'Cambia! Volatility', 'port_rets': 'Expected Return',
            'sharpe_ratio': 'Sharpe Ratio'},
            title='Monte Carlo Simulated Portfolio'
).update_traces(mode='markers',marker=dict(symbol='cross'))

# Plot max sharpe
fig.add_scatter(
    mode='markers',
    x=[portfolioSamples.iloc[portfolioSamples.sharpe_ratio.idxmax()]['port_vols']],
    y=[portfolioSamples.iloc[portfolioSamples.sharpe_ratio.idxmax()]['port_rets']],
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe',
).update(layout_showlegend=False)

fig.add_scatter(
    x=partial_statistics["Average annual volatility"],
    y=partial_statistics["Average annual return"],
    text = [f'Asset: {index}<br>Volatility: {vol:.2f}<br>Return: {ret:.2f}' 
            for index, vol, ret in zip(partial_statistics.index, partial_statistics["Average annual volatility"], partial_statistics["Average annual return"])],
    hoverinfo='text',
    mode='markers',
    marker=dict(color='green', size=8),
)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [10]:
return_lowerBound, return_upperBound = 0.1, 0.11
portfolioSamples[(portfolioSamples['port_rets']>return_lowerBound) & (portfolioSamples['port_rets']<=return_upperBound)]


,port_rets,port_vols,ENI.MI weight,LDO.MI weight,STMMI.MI weight,TRN.MI weight,UCG.MI weight,sharpe_ratio
10,0.1010,0.2669,0.2321,0.1856,0.2249,0.2142,0.1431,0.3785
17,0.1061,0.2682,0.1066,0.2391,0.2089,0.2907,0.1547,0.3954
19,0.1007,0.2968,0.3130,0.3314,0.2710,0.0161,0.0684,0.3392
25,0.1006,0.3188,0.1628,0.3118,0.3281,0.0026,0.1947,0.3155
29,0.1100,0.3328,0.0249,0.4121,0.3398,0.0348,0.1884,0.3304
...,...,...,...,...,...,...,...,...
9964,0.1020,0.3099,0.1241,0.0146,0.3584,0.1535,0.3494,0.3292
9975,0.1042,0.2641,0.0175,0.3264,0.1221,0.4092,0.1248,0.3946
9981,0.1052,0.2964,0.3732,0.0940,0.3623,0.0056,0.1650,0.3549
9986,0.1019,0.2716,0.0538,0.1638,0.2109,0.3341,0.2374,0.3754


In [11]:
index_maxVol = portfolioSamples[(portfolioSamples['port_rets']>return_lowerBound) & (portfolioSamples['port_rets']<=return_upperBound)].port_vols.idxmax()
index_minVol = portfolioSamples[(portfolioSamples['port_rets']>return_lowerBound) & (portfolioSamples['port_rets']<=return_upperBound)].port_vols.idxmin()

In [12]:
portfolio_index = 774
portfolioSamples.iloc[portfolio_index]

port_rets          0.1129
port_vols          0.2202
ENI.MI weight      0.2466
LDO.MI weight      0.1134
STMMI.MI weight    0.0978
TRN.MI weight      0.5161
UCG.MI weight      0.0261
sharpe_ratio       0.5126
Name: 774, dtype: float64

In [13]:
column_weights = [x+' weight' for x in yahooticker]
weights = portfolioSamples.iloc[portfolio_index][column_weights].to_numpy()

In [14]:
indices = [index_minVol, index_maxVol,]
weights_set = [portfolioSamples.iloc[index][column_weights].to_numpy() for index in indices]


In [15]:
def evaluateStock(stock,source,start,end = 'today'):
    end_date = datetime.today().strftime('%Y-%m-%d') if (end == 'today') else end
    return source.loc[start:end_date,stock]

def evaluateMarket(source,start,end = 'today'):
    return pd.DataFrame({stock: evaluateStock(stock, source, start, end) for stock in source.columns})

def evaluatePortfolio(source, weights, initialWealth, start, end = 'today'):
    stockPrices = evaluateMarket(source,start,end)
    nShares = weights*initialWealth/stockPrices.iloc[0].to_numpy()
    portfolio_df = pd.DataFrame({
        'Date': stockPrices.index,
        'portfolioValue': np.dot(stockPrices.to_numpy(), nShares)
    })
    portfolio_df.set_index('Date',inplace=True)
    return portfolio_df
    
def evaluateMultiplePortfolios(source,weights_set, initialWealth, start, end = 'today'):
    dfs = [evaluatePortfolio(source, wts, initialWealth, start, end) for wts in weights_set]
    df = pd.DataFrame({'Portfolio_'+str(i+1): dfs[i]['portfolioValue'] for i in range(len(dfs))})
    return df

In [20]:

evaluateMultiplePortfolios(df,weights_set, 100, '2021', 'today').iplot()

In [17]:
evaluatePortfolio(df,weights,100,'2021','2021')

,portfolioValue
Date,
2021-01-04,100.0000
2021-01-05,100.6583
2021-01-06,102.5645
2021-01-07,102.5014
2021-01-08,102.2668
...,...
2021-12-23,130.0043
2021-12-27,130.3749
2021-12-28,131.0170


In [23]:
evaluatePortfolio(df,weights,100,'2021','2021').iplot(asUrl = True, filename = 'html_plot',theme='white')


'html_plot.html'

In [19]:
portfolioRealizedAnnualReturn = evaluatePortfolio(df,weights,100,'2021').pct_change().dropna().mean()*tradingDaysPerYear
portfolioRealizedAnnualVolatility = evaluatePortfolio(df,weights,100,'2021').pct_change().dropna().std()*np.sqrt(tradingDaysPerYear)

print(f"{portfolioRealizedAnnualReturn = }\n{portfolioRealizedAnnualVolatility =}")

portfolioRealizedAnnualReturn = portfolioValue    0.2091
dtype: float64
portfolioRealizedAnnualVolatility =portfolioValue    0.1699
dtype: float64


Evaluate single assets performance. Evaluate portfolio performance both in absolute terms and in returns/std. compare with the expected values based on historical data. Write functions to perform such computations for all portfolios. Give conclusion on markowitz theory. Use more data?

In [26]:
(1.1)**3

1.3310000000000004